<a href="https://colab.research.google.com/github/LucasBezerraSantos/Projetos_Pessoias/blob/master/MLOps_Estudos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MLOps

Este projeto tem o objetivo de reunir conceitos básico de MLOps para permitir que os Modelos sejão consumidos e compartilhados através de APIs.

*   Os passos envolvem criação de API
*   Conceito de Endpoint
*   Consumo de APIs via postman, JupyterLab e WEB
*   Autenticação
*   Serialização
*   Criação de Maquinas Virtuais e Controle de Dependências


## Modelo simples de Regressão Linear para aplicar na API

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from google.colab import files
import io

OBS. *A biblioteca "io" em Python é utilizada para trabalhar com operações de entrada e saída (I/O), oferecendo classes e funções para manipular streams e dados de diferentes fontes, como arquivos e strings.*

In [19]:
uploaded = files.upload() # leitura de arquivos

Saving casas.csv to casas (1).csv


In [22]:
df = pd.read_csv(io.BytesIO(uploaded['casas (1).csv'])) # conversão em DF
df.head()

,tamanho,ano,garagem,preco
0,159.0,2003,2,208500
1,117.0,1976,2,181500
2,166.0,2001,2,223500
3,160.0,1915,3,140000
4,204.0,2000,3,250000


In [32]:
X = df.drop('preco', axis=1)
y = df.preco

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.30, random_state=42 )

modelo_linear = LinearRegression()
modelo_linear.fit(X_train, y_train)

resultado = modelo_linear.predict([[120, 2001, 2]]) # teste
print('Preço do Imóvel: ', round(*resultado))

Preço do Imóvel:  191437


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


**Serialização**

É possível salvar o Modelo em um arquivo e reutiliza-lo em outra IDE ou ambiente de codificação. Isso aumenta a produtividade pois elimina a necessidade de ter que transportar o código inteiro para outra IDE.

In [13]:
import pickle
pickle.dump(modelo_linear, open('modelo.sav', 'wb'))

In [15]:
files.download('modelo.sav') # gera um arquivo com o modelo salvo

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

OBS. *A biblioteca "pickle" em Python é usada para serializar e desserializar objetos Python. Ela converte objetos complexos em uma representação binária para armazenamento em disco ou transmissão pela rede, permitindo a recuperação dos dados com sua estrutura e estado originais. É útil em caching, criação de pontos de salvamento, compartilhamento de dados entre aplicativos ou componentes, e ambientes de computação distribuída e paralela, mas requer cuidado com dados não confiáveis devido a possíveis riscos de segurança na desserialização.*

## Criando uma API

*Uma API (Interface de Programação de Aplicativos) é um conjunto de regras e protocolos que permite que diferentes aplicativos e sistemas interajam e se comuniquem entre si. Ela define os métodos e formatos de solicitação e resposta que os desenvolvedores podem usar para acessar e utilizar os serviços ou funcionalidades oferecidas por uma aplicação, serviço web ou plataforma. As APIs atuam como intermediárias, permitindo que aplicativos se conectem de maneira padronizada, facilitando a integração e a troca de dados.*

In [ ]:
# para construção da API foi utilizado o Visual Studio Code

from flask import Flask, request, jsonify
import pickle
from flask_basicauth import BasicAuth

```
Flask: Uma biblioteca Python para criação de aplicativos web, permitindo a construção de APIs, sites e outros serviços web de forma rápida e simples.

Request: Uma parte do Flask que permite acessar os dados enviados pelo cliente ao servidor, como formulários web ou parâmetros de URL, facilitando o tratamento desses dados no aplicativo.

Jsonify: Uma função do Flask que converte objetos Python em objetos JSON, útil para retornar respostas em formato JSON a partir do aplicativo web.

Pickle: Uma biblioteca para serialização e desserialização de objetos Python, permitindo salvar e carregar estruturas de dados de forma eficiente.

Flask-BasicAuth: Uma extensão do Flask que fornece autenticação básica para aplicativos web, permitindo controlar o acesso a certas rotas ou recursos com autenticação de usuário e senha.
```



In [ ]:
colunas = ['tamanho', 'ano', 'garagem']           # lista com colunas da variável X do modelo
modelo = pickle.load(open('modelo.sav', 'rb'))    # para abrir o modelo salvo com a Regressão Linear | deve estar no mesmo diretorio do VScode

app = Flask(__name__)                             # define o nome para API
app.config['BASIC_AUTH_USERNAME'] = 'escolhaonome'
app.config['BASIC_AUTH_PASSWORD'] = 'escolha a senha'

basic_auth = BasicAuth(app)                       # autenticação simples

@app.route('/')                                   # endpoint: cria uma rota para a API chamada de "app"
@basic_auth.required                              # solicita a verificação conforme autenticação
def home():
    return "minha primeira API"                   # o que a API executa ao ser chamada

@app.route('/modelo/', methods=['POST'])          # endpoint: usa método POST para receber variáveis de entrada
@basic_auth.required
def executa_modelo():
    dados = request.get_json()
    dados_input = [dados[col] for col in colunas] # ordenada variaveis de entrada conforme colunas definidas
    preco = modelo.predict([dados_input])
    return jsonify(preco=preco[0])                # devolve a resposta no formato Json

app.run(debug=True)

OBS. *Em APIs, os endpoints são os pontos de acesso específicos aos recursos ou serviços disponibilizados pelo servidor. Cada endpoint é associado a uma URL única que os clientes da API podem usar para interagir com o servidor e realizar operações sobre os dados ou executar ações específicas. Os endpoints definem as diferentes funcionalidades da API e geralmente são usados para executar operações CRUD (Criar, Ler, Atualizar, Excluir) em recursos. Por exemplo, em uma API de um aplicativo de mídia social, poderia haver endpoints como "/usuarios" para obter informações de usuários, "/publicacoes" para recuperar postagens e "/amigos" para gerenciar conexões entre usuários. Os endpoints permitem que aplicativos externos se comuniquem com o servidor de forma padronizada e bem definida, facilitando a integração e o acesso aos recursos oferecidos pela API.*

## Consumindo uma API pelo Jupyterlab

In [ ]:
import requests

#definindo 3 variáveis:

url = 'http://linkdaAPI/'

dados_exemplo = {
    "tamanho": 220,
    "ano": 2001,
    "garagem": 2
}

auth = requests.auth.HTTPBasicAuth('nome', 'senha')

response = requests.post( url, json=dados_exemplo, auth=auth )

#response. contém variáveis de acesso aos dados da API

## Prompt de Comando

Após esta etapa é preciso abrir o Prompt de Comando e localizar o diretório onde o arquivo do VS-CODE com o código está salvo. Depois chamar: python "nomedoarquivo.py"

Para que seja executado corretamente, todas as bibliotécas devem estar instaladas com a mesma versão que estão no VS-CODE e na criação do modelo.

O Prompt vai gerar a URL com o código da maquina, onde será possível consultar a API na WEB.

In [ ]:
# Usando o POSTMAN as configurações abaixo permitem acessar o modelo através da API

#   POSTMAN
#   POST
#   BODY: ROW
#   {
#       "tamanho": 220,
#        "ano": 2001,
#       "garagem": 2
#   }

#   HEADERS: Content-Type
#   VALUE: application/json

## Máquina Virtual com Virtualenv

Para criar uma máquina virtual é possível usar a bibliotéca Python Virtualenv, inserindo no Prompt: **C:\>python -m virtualenv "nomedamaquinavirtual"**

---

Para reunir todas as versões de Bibliotecas usadas Prompt use: **pip freeze > requirements.txt** (esse comando cria um arquivo txt no diretório com todas as bibliotécas e versões utilizadas no Virtualenv)

---

Para instalar todas as bibliotécas em novo ambiente virtual, use: **pip install -r requirements.txt**

OBS. Importante a máquina virtual estar dentro do mesmo diretório do projeto para que possa executar com o comando **python "nomedoarquivo.py"**

